# Install Roboflow

In [1]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstalled opencv-python-headless-4.8.1.78
  Attempting uninstall: idna
    Found existing

# Infer and Generate JSON File

In [3]:
from roboflow import Roboflow
import json

rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("tim-4ijf0").project("plantmo")
model = project.version(1).model

# infer on a local image
print(model.predict("/content/50.jpg", confidence=80, overlap=50).json())
predictions_json = model.predict("50.jpg", confidence=40, overlap=30).json()

output_json_path = '/content/detection_results.json'

# Save the predictions to a JSON file
with open(output_json_path, 'w') as json_file:
    json.dump(predictions_json, json_file, indent=2)

print(f"Predictions saved to {output_json_path}")

loading Roboflow workspace...
loading Roboflow project...
{'predictions': [{'x': 428.0, 'y': 294.5, 'width': 280.0, 'height': 395.0, 'confidence': 0.9443135261535645, 'class': 'plant', 'class_id': 0, 'image_path': '/content/50.jpg', 'prediction_type': 'ObjectDetectionModel'}, {'x': 86.0, 'y': 450.5, 'width': 26.0, 'height': 137.0, 'confidence': 0.8782252073287964, 'class': 'ref_obj', 'class_id': 1, 'image_path': '/content/50.jpg', 'prediction_type': 'ObjectDetectionModel'}], 'image': {'width': '800', 'height': '800'}}
Predictions saved to /content/detection_results.json


# Calculate Plant Height

In [4]:
# Final Code
import json
import cv2
import numpy as np

def calculate_pixel_height(bbox):
    return bbox['height']

def establish_conversion_factor(reference_height, reference_pixel_height):
    return reference_height / reference_pixel_height

def process_object_detection_results(json_path, image_path, output_path):
    # Load the JSON results
    with open(json_path, 'r') as file:
        detection_results = json.load(file)

    # Extract bounding box information for the reference object and the plant
    reference_bbox = next(item for item in detection_results['predictions'] if item['class'] == 'ref_obj')
    plant_bbox = next(item for item in detection_results['predictions'] if item['class'] == 'plant')

    # Calculate pixel height of the reference object
    reference_pixel_height = calculate_pixel_height(reference_bbox)

    # Known height of the reference object in inches
    reference_height_inches = 6

    # Establish pixel-to-inch conversion factor
    conversion_factor = establish_conversion_factor(reference_height_inches, reference_pixel_height)

    # Calculate the actual height of the plant in inches
    plant_pixel_height = calculate_pixel_height(plant_bbox)
    plant_height_inches = plant_pixel_height * conversion_factor

    # Load the original image
    image = cv2.imread(image_path)

    # Draw bounding boxes and labels on the image
    xr = reference_bbox['x'] - (reference_bbox['width']/2);
    yr = reference_bbox['y'] - (reference_bbox['height']/2)
    reference_box = [

        round(xr),
        round(yr),
        round(xr + reference_bbox['width']),
        round(yr + reference_bbox['height'])
    ]

    xp = plant_bbox['x'] - (plant_bbox['width']/2);
    yp = plant_bbox['y'] - (plant_bbox['height']/2);
    plant_box = [
        round(xp),
        round(yp),
        round(xp + plant_bbox['width']),
        round(yp + plant_bbox['height'])
    ]

    # Draw reference object bounding box in green
    cv2.rectangle(image, (reference_box[0], reference_box[1]), (reference_box[2], reference_box[3]), (0, 255, 0), 2)

    # Draw plant bounding box in red
    cv2.rectangle(image, (plant_box[0], plant_box[1]), (plant_box[2], plant_box[3]), (0, 0, 255), 2)

    # Display labels
    font = cv2.FONT_HERSHEY_SIMPLEX
    label_y_offset = 20
    cv2.putText(image, f"Reference Object Height: {reference_height_inches} inches", (reference_box[0], reference_box[1] - label_y_offset),
                font, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.putText(image, f"Plant Height: {plant_height_inches:.2f} inches", (plant_box[0], plant_box[1] - label_y_offset),
                font, 0.5, (0, 0, 255), 2, cv2.LINE_AA)

    # Save the annotated image
    cv2.imwrite(output_path, image)
    print(f"Annotated image saved to {output_path}")

# Specify the path to your JSON file, the original image, and the output image
json_path = '/content/detection_results.json'
image_path = '/content/50.jpg'
output_image_path = '/content/output_annotated_50.jpg'

# Call the function to process object detection results, calculate heights, and save annotated image
process_object_detection_results(json_path, image_path, output_image_path)


Annotated image saved to /content/output_annotated_50.jpg
